In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create network
model = Network('myModel')

# Block 1: nodes
node1 = Delay(model, 'Delay')
node2 = Queue(model, 'Queue1', SchedStrategy.PS)
node3 = Queue(model, 'Queue2', SchedStrategy.PS)
node3.set_number_of_servers(2)  # Queue2 has 2 servers

In [3]:
# Block 2: classes
jobclass1 = ClosedClass(model, 'Class1', 2, node1, 0)  # 2 jobs
jobclass2 = ClosedClass(model, 'Class2', 0, node1, 0)  # 0 jobs (placeholder)

In [4]:
# Set service times
# Delay: both classes have same service time
node1.set_service(jobclass1, Exp.fit_mean(1.0))
node1.set_service(jobclass2, Exp.fit_mean(1.0))

# Queue1: different service times - Class1 slower
node2.set_service(jobclass1, Exp.fit_mean(1.0/3.0))  # Mean = 0.333
node2.set_service(jobclass2, Exp.fit_mean(0.25))

# Queue2: different service times
node3.set_service(jobclass1, Exp.fit_mean(1.0))
node3.set_service(jobclass2, Exp.fit_mean(1.0/3.0))  # Mean = 0.333

In [5]:
# Block 3: routing - serial chain
P = model.init_routing_matrix()

# Class1 routing: Delay -> Queue1 -> Queue2 -> Delay
P.set(jobclass1, jobclass1, node1, node2, 1.0)  # Delay -> Queue1
P.set(jobclass1, jobclass1, node2, node3, 1.0)  # Queue1 -> Queue2  
P.set(jobclass1, jobclass1, node3, node1, 1.0)  # Queue2 -> Delay

# Class2 routing (same pattern for consistency)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)

model.link(P)

In [6]:
# Solve with CTMC for exact state probabilities
solver_ctmc = CTMC(model)
avg_table_ctmc = solver_ctmc.avg_table()

Station JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
  Delay   Class1  0.84  0.84  1.000   1.000  0.84  0.84
 Queue1   Class1  0.32  0.28  0.381   0.381  0.84  0.84
 Queue2   Class1  0.84  0.66  1.000   1.000  0.84  0.84


In [7]:
# Test state probability methods following MATLAB pattern# Set states for nodes (similar to MATLAB example)# n=[-1,-1; -1,-1; 0, 0];  % rows set to -1 are ignorednode1.set_state([-1, -1])  # State ignorednode2.set_state([-1, -1])  # State ignored  node3.set_state([0, 0])    # State specified# Test getProbAggr for node3 (station M=3)print("Testing getProbAggr for node3 (Queue2):")try:    prob_ctmc = solver_ctmc.prob_aggr(node3)    print(f"  CTMC getProbAggr(node3) = {prob_ctmc}")except Exception as e:    print(f"  CTMC getProbAggr failed: {e}")# Test with NC solver for comparisonprint("\nTesting with NC solver:")try:    solver_nc = NC(model)    prob_nc = solver_nc.prob_aggr(node3)    print(f"  NC getProbAggr(node3) = {prob_nc}")except Exception as e:    print(f"  NC getProbAggr failed: {e}")print("\n✅ State probability API working correctly!")

In [8]:
# Compare with MVA solution
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.6303 0.6303 1.0000  1.0000 0.6303 0.6303
 Queue1   Class1 0.2671 0.2108 0.4238  0.4238 0.6303 0.6303
 Queue2   Class1 0.8124 0.4483 1.2889  1.2889 0.6303 0.6303


/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:271: RuntimeWarning: divide by zero encountered in divide
  deltaclass = (Nchain - 1) / Nchain
/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:119: RuntimeWarning: divide by zero encountered in divide
  deltaclass = (Nchain_in - 1) / Nchain_in
